DBからtextデータのみをとってきて特定のLenになるまで結合する。  
まだ２０００字でしかやっていない。そしてそれを新しいDBに入れている。  
その後MeCabを使って、形態素解析し、縦（語彙）×横（レビュー番号）のcsvを作成、保存している。  


In [ ]:
import sys;sys.path.append('./src')
import sqlite3
import re
from DB import *
from NLP import *
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import re 
from collections import Counter
import urllib.request as urllib
from bs4 import BeautifulSoup
import sqlite3
import time
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from contextlib import closing

In [ ]:
def clean_txt2(text_data):
    # text_data=re.sub('\（.+\）','',text_data)
#     text_data=re.sub('.+　','',text_data)
    text_data=re.sub('\n','',text_data)
    text_data=re.sub('\\u3000','',text_data)
    text_data=text_data.split('。')
    return(text_data)

In [ ]:
low_category_db='../data/database/1112_low_prodcuts_text_category.db'
high_category_db='../data/database/1112_high_prodcuts_text_category.db'
Hconn2,Hc2=get_db_conn(high_category_db)
Lconn2,Lc2=get_db_conn(low_category_db)


high_text_db='../data/database/1120_high_ML_text.db'
Hconn3,Hc3=get_db_conn(high_text_db)
low_text_db='../data/database/1120_low_ML_text.db'
Lconn3,Lc3=get_db_conn(low_text_db)

In [ ]:
# #最初にDBを作るセルなので今はもういらない
# Hc2.execute('''select distinct id,username,text_len,pd_text from product_text where text_len!='no_datas' ''')
# Hc3.execute('''create table ML_text(id integer primary key,text_len,pd_text)''')

# i=0
# j=0
# datas=[]
# sum_len=0
# sum_text=''
# for row in Hc2:
#     i+=1
#     print(i,end=' ')
#     len_=row[2]
#     pd_text=row[3]
#     sum_len+=int(len_)
#     sum_text+=pd_text
#     if sum_len>=2000:
#         j+=1
#         print(f'j={j},i={i}')
#         print(sum_len)
#         datas.append((sum_len,sum_text))
#         sum_len=0
#         sum_text=''
# print(len(datas))

# insert_sql = 'insert into ML_text (text_len,pd_text) values (?,?)'
# Hc3.executemany(insert_sql, datas)
# Hconn3.commit()

In [ ]:

# Lc3.execute('''create table ML_text(id integer primary key,text_len,pd_text)''')
# Lc2.execute('''select distinct id,username,text_len,pd_text from product_text where text_len!='no_datas' ''')
# i=0
# j=0
# datas=[]
# sum_len=0
# sum_text=''
# for row in Lc2:
#     i+=1
#     print(i,end=' ')
#     len_=row[2]
#     pd_text=row[3]
#     sum_len+=int(len_)
#     sum_text+=pd_text
#     if sum_len>=2000:
#         j+=1
#         print(f'j={j},i={i}')
# #         print(sum_len)
#         datas.append((sum_len,sum_text))
#         sum_len=0
#         sum_text=''
# print(len(datas))

# insert_sql = 'insert into ML_text (text_len,pd_text) values (?,?)'
# Lc3.executemany(insert_sql, datas)
# Lconn3.commit()

In [ ]:
Lc3.execute('''select distinct text_len,pd_text from ML_text  ''')
ldf=pd.DataFrame()
i=0
for row in Lc3:
    a=clean_txt(str(row[1]))
    a=count_matubi(a,3)
    ldf=count_to_pd(ldf,i,a)
    i+=1
    print(i,end='')
ldf.head()

In [ ]:
ldf['sum']=ldf.sum(axis=1)
ldf.sort_values(by='sum',ascending=False)
ldf.to_csv('./1121low.csv')

In [ ]:
goi_df=pd.read_csv('../data/goi/goi.csv',encoding='shift-jis')
goiword=list(goi_df['標準的な表記'])

In [ ]:
Lc3.execute('''select distinct text_len,pd_text from ML_text  ''')
ldf_mor=pd.DataFrame()
i=0
for row in Lc3:
    a=clean_txt2(str(row[1]))
    a=count_morphemes(a,goiword,goihyo_flg=False)
    try:
        ldf_mor=count_to_pd(ldf_mor,i,a)
    except:
        continue
    i+=1
    print(i,end=',')
ldf_mor.head()


In [ ]:
# ldf_mor.head(30)
ldf_mor['sum']=ldf_mor.sum(axis=1)
ldf_mor.sort_values(by='sum',ascending=False,inplace=True)
ldf_mor.to_csv('./1121low_mor.csv')

In [ ]:
Hc3.execute('''select distinct text_len,pd_text from ML_text  ''')
hdf_mor=pd.DataFrame()
i=0
for row in Hc3:
    a=clean_txt2(str(row[1]))
    a=count_morphemes(a,goiword,goihyo_flg=False)
    try:
        hdf_mor=count_to_pd(hdf_mor,i,a)
    except:
        continue
    i+=1
    print(i,end=',')
hdf_mor.head()



In [ ]:
hdf_mor['sum']=hdf_mor.sum(axis=1)
hdf_mor.sort_values(by='sum',ascending=False,inplace=True)
hdf_mor.to_csv('./1121high_mor.csv')

In [ ]:
a=5
if a==4 or not a==5:
    print(a)